In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_df = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
train_df.shape , test_df.shape

In [4]:
test_df.columns

In [5]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor , Lambda

class CustomMNISTDataset(Dataset):
    def __init__(self, csv_name, img_dir, transform=None, target_transform=None , label_name = "label"):
        
        self.img_filename = csv_name
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.label_name = label_name
        
        img_path = os.path.join(self.img_dir, self.img_filename)
        self.img_df = pd.read_csv(img_path)

    def __len__(self):
        return len(self.img_df)

    def __getitem__(self, idx):
        
        # Extracting all the other columns except label_name
        img_cols = [ i for i in self.img_df.columns if i not in self.label_name]
        
        image = self.img_df.iloc[[idx]][img_cols].values
        label = int(self.img_df.iloc[[idx]][self.label_name].values)
        
        # Reshaping the array from 1*784 to 28*28
        
        # image = image.reshape(28,28)
        # image = image.astype(float)
        
        # Scaling the image so that the values only range between 0 and 1
        # image = image/255.0
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        
        image = image.to(torch.float)
        label = label.to(torch.float)
        
        return image, label

In [6]:
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
train_df['label'].value_counts().sort_index()

In [7]:
## Illustration of creating a validation set 

from sklearn.model_selection import train_test_split
indices = list(range(len(train_df)))

train_indices , test_indices = train_test_split(indices, test_size=0.1, stratify=train_df['label'])
# train_indices , test_indices = train_test_split(indices, test_size=0.1)

len(train_indices) , len(test_indices) , len(train_df)

train_subset = train_df.loc[train_indices]
val_subset = train_df.loc[test_indices]

print( train_subset['label'].value_counts().sort_index() / train_subset['label'].value_counts().sort_index().sum() )
print( val_subset['label'].value_counts().sort_index() / val_subset['label'].value_counts().sort_index().sum() )

In [8]:

# Crerating a temp dataset
train_csv_name = "train.csv"
test_csv_name = "test.csv"
img_dir = "/kaggle/input/digit-recognizer/"

# Converting X variables to Tensors
transforms = ToTensor()

# Converting y-labels to one hot encoding
target_transform = Lambda(lambda y: torch.zeros(
    len(train_df['label'].unique()), dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

label_name = "label"

train_dataset = CustomMNISTDataset(csv_name = train_csv_name , img_dir = img_dir , transform = transforms , target_transform = target_transform , label_name = label_name)
# test_dataset = CustomMNISTDataset(csv_name = test_csv_name , img_dir = img_dir , transform = transforms , target_transform = target_transform , label_name = label_name)
x0 , y0 = train_dataset[0]
print(x0.shape , y0.shape)

In [9]:
## Checking if the GPU is being used properly . 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
torch.cuda.is_available()
x0 = x0.to(device)
print("x0" , x0.is_cuda)

In [10]:
torch.cuda.get_device_name(0)

In [15]:
from torch.utils.data import SubsetRandomSampler
from sklearn.model_selection import train_test_split
indices = list(range(len(train_df)))
train_indices , test_indices = train_test_split(indices, test_size=0.1, stratify=train_df['label'])

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_dataloader = torch.utils.data.DataLoader(train_dataset , batch_size=64, sampler=train_sampler, num_workers=16)
test_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, sampler=test_sampler, num_workers=16)

In [16]:
x0 , y0 = next(iter(train_loader))
x0.shape , y0.shape

In [40]:
# Creating data loaders using temp_dataset
from torch.utils.data import DataLoader

# train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [17]:
from torch import nn

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class MyOwnNeuralNetwork(nn.Module):
    def __init__(self):
        super(MyOwnNeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return torch.softmax(logits , dim = 1)
    
model = MyOwnNeuralNetwork().to(device)
print(model)

# model = model.cuda()
# torch.backends.cudnn.benchmark=True
# torch.cuda.set_device(0)

In [18]:
## Defining optimizer and loss functions 

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [25]:
from torch.autograd import Variable
def train(dataloader, model, loss_fn, optimizer):
    
    # Total size of dataset for reference
    size = len(dataloader.dataset)
    
    # places your model into training mode
    model.train()
    
    # Gives X , y for each batch
    for batch, (X, y) in enumerate(dataloader):
        
        # Converting device to cuda
        X, y = X.to(device), y.to(device)
        model.to(device)
        
        # print("X : " , X.is_cuda , " , y : ", y.is_cuda )
        
        # dtype = torch.cuda.FloatTensor
        # X = Variable(X).type(dtype)
        # y = Variable(y).type(dtype)

        # Compute prediction error / loss
        # 1. Compute y_pred 
        # 2. Compute loss between y and y_pred using selectd loss function
        
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        # Backpropagation on optimizing for loss
        # 1. Sets gradients as 0 
        # 2. Compute the gradients using back_prop
        # 3. update the parameters using the gradients from step 2
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
data = next(iter(test_dataloader))
data[0].shape , data[1].shape

In [50]:
data[1]

In [44]:
data = next(iter(train_dataloader))
data[0].shape , data[1].shape

In [21]:
def test(dataloader, model, loss_fn):
    
    # Total size of dataset for reference
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    # Explanation given above
    model.eval()

    test_loss, correct = 0, 0
    with torch.no_grad():
        
        # Gives X , y for each batch
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            model.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    
    ## Calculating loss based on loss function defined
    test_loss /= num_batches
    
    ## Calculating Accuracy based on how many y match with y_pred
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")